In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local[*]") \
        .appName('test') \
        .getOrCreate()

22/03/01 13:01:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df_green = spark.read.parquet("data/pq/green/*/*")

In [3]:
df_green.registerTempTable('green')

In [4]:
df_green.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [5]:
df_green_revenue = spark.sql("""
Select 
    date_trunc('hour',lpep_pickup_datetime) as hour,
    PULocationID as zone,
    
    SUM(total_amount) as amount,
    Count(1) as number_records
from 
    green
WHERE 
    lpep_pickup_datetime > '2020-01-01'
GROUP BY 
    1,2
""")

In [6]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green',mode='overwrite');

In [7]:
df_yellow = spark.read.parquet("data/pq/yellow/*/*")

In [8]:
df_yellow.registerTempTable('yellow')

In [9]:
df_yellow_revenue = spark.sql("""
Select 
    date_trunc('hour',tpep_pickup_datetime) as hour,
    PULocationID as zone,
    
    SUM(total_amount) as amount,
    Count(1) as number_records
from 
    yellow
WHERE 
    tpep_pickup_datetime > '2020-01-01'
GROUP BY 
    1,2
""")

In [10]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow',mode='overwrite')

In [19]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount','green_amount') \
    .withColumnRenamed('number_records','green_number_records') 

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount','yellow_amount') \
    .withColumnRenamed('number_records','yellow_number_records') 


In [20]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour','zone'],how='outer')

In [21]:
df_join.write.parquet('data/report/revenue/total')

In [22]:
df_join = spark.read.parquet('data/report/revenue/total')

In [25]:
df_zones = spark.read.parquet('zones/')

In [27]:
df_result = df_join.join(df_zones,df_join.zone == df_zones.LocationID)

In [30]:
df_result.drop('LocationID','zone').write.parquet('tmp/revenue-zones')

In [23]:
df_join.show()

+-------------------+----+------------+--------------------+------------------+---------------------+
|               hour|zone|green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------+--------------------+------------------+---------------------+
|2020-01-01 02:00:00| 163|        null|                null| 4672.539999999999|                  249|
|2020-01-01 03:00:00|  66|       65.66|                   4|             81.05|                    3|
|2020-01-01 03:00:00| 114|        null|                null|6185.9600000000055|                  310|
|2020-01-01 05:00:00| 146|        null|                null|344.78000000000003|                    9|
|2020-01-01 08:00:00| 136|        null|                null|             132.3|                    2|
|2020-01-01 10:00:00| 200|        null|                null|              38.6|                    1|
|2020-01-01 12:00:00|  37|        null|                null|               7.8|   